In [4]:
import os
import numpy as np
import pandas as pd

from astropy.io import fits
from astropy.wcs import WCS
from astropy.stats import mad_std
from astropy.wcs.utils import pixel_to_skycoord

from photutils import CircularAperture, CircularAnnulus, SkyCircularAperture, aperture_photometry, DAOStarFinder, IRAFStarFinder

from a345_utilities import print_header, mem_usage     # to use this in your own scripts copy the file a345_utilities.py to your work directory

In [7]:
#Cluster directories:
M35_dir = 'M35'
M67_dir = 'M67'
M67v2_dir = 'M67v2'
NGC869_dir = 'NGC869'

#Setting Cluster to process
cluster_dir = NGC869_dir  #------------------------------------------!!!!!

#Output directories to shared and local folders
shared_dir = os.path.join('/data/observatory/student_data/William_ODonnell', cluster_dir)
local_dir = cluster_dir

In [8]:
Coord_df = pd.read_csv(os.path.join(local_dir, '1. '+ cluster_dir + '_Coord_data.csv'))

R = (np.median(Coord_df['Right-Ascension'])) + (np.std(Coord_df['Right-Ascension']))/4 #Max value: 1/4 s.d. above the median
D = (np.median(Coord_df['Declination'])) + (np.std(Coord_df['Declination']))/4 #Max value: 1/4 s.d. above the median

for n in range(len(Coord_df['Right-Ascension'])): #Removing anything 1/4 s.d. above median (somehow wrong coords)
    if Coord_df['Right-Ascension'][n] > R or Coord_df['Declination'][n] > D:
        Coord_df = Coord_df.drop([n])
        
Coord_df = Coord_df.round({'Right-Ascension': 3, 'Declination': 3})
ra = Coord_df['Right-Ascension']
dec = Coord_df['Declination']

G_ID = []; G_RA = []; G_DEC = []; G_Flux = []
I_ID = []; I_RA = []; I_DEC = []; I_Flux = []
R_ID = []; R_RA = []; R_DEC = []; R_Flux = []


for file in os.listdir(shared_dir):
    if file.endswith(".fits"):
        print("----------------------------------------------------------------------")
        print("----------------------------------------------------------------------")
        print("Opening file: ", file)
        with fits.open(os.path.join(shared_dir, file)) as hdu:
            header = hdu[0].header
            data = hdu[0].data
            
        wcs = WCS(header)
        name = os.path.splitext(file)[0]

###The Photometry!!!!
#This code is applied to all images 
    
    #This block finds the brightest sources in each image and averages the fhwm of them
        bkg_sigma = mad_std(data)       
        SeeingStars = IRAFStarFinder(fwhm=8, threshold=50*bkg_sigma, brightest=500)  
        SeeingStars = SeeingStars(data)
        fwhm = np.mean(SeeingStars['fwhm']) #calculates the average size of the stars in the image

    #converts the ra, dec of 'known' stars to pixels
        xc,yc = wcs.wcs_world2pix(ra, dec, 1)
        PixPos = np.transpose((xc,yc))

      
    # sets the aperature and radii to use
        pix_radius = 4*fwhm/2 # 4 times the average star raduis to collect all the possible photons
        In_ann = pix_radius*1.2; Out_ann = (In_ann+1)*1.3 # annulus measurements from 1.2*aperture to 1.3*aperture

    # Setting the aperture and annulus radii over each source based on the pixel coords converted from the RA-DEC list of the master.
        aperture = CircularAperture(PixPos, r=pix_radius) # creating the cicular apertures cnetred on the star positions
        annulus_aperture = CircularAnnulus(PixPos, r_in=In_ann, r_out=Out_ann) # creating the annulus apertures cnetred on the star positions
        apers = [aperture, annulus_aperture]

    # Send the created apertures and the calibrated data to do the photometry
        print('Starting aperture photometry...')
        phot_table = aperture_photometry(data, apers) #The actual photometry to find the counts for aperture area and annulus area

     # Once The Photometry has been done for each source
        bkg_mean = phot_table['aperture_sum_1'] / annulus_aperture.area # average value of the annulus around each star (BACKGROUND COUNT)
        phot_table['background_mean'] = bkg_mean                        # adding the background level to the output table (using the mean instead of median due to cloud blurring)
        ID = phot_table['id']
        bkg_sum = bkg_mean * aperture.area                              # background for each star
        final_sum = phot_table['aperture_sum_0'] - bkg_sum              # The actual photometry results
        filter_flux_name = 'flux_'+header['FILTER']                      
        filter_flux = final_sum/header['EXPTIME']


    #Saving table directly to a text file
        if header['FILTER'] == "G'":
            G_ID.extend(ID)
            G_RA.extend(ra)
            G_DEC.extend(dec)
            G_Flux.extend(filter_flux)
        elif header['FILTER'] == "R'":
            R_ID.extend(ID)
            R_RA.extend(ra)
            R_DEC.extend(dec)
            R_Flux.extend(filter_flux)
        elif header['FILTER'] == "I'":
            I_ID.extend(ID)
            I_RA.extend(ra)
            I_DEC.extend(dec)
            I_Flux.extend(filter_flux)
print("----------------------------------------------------------------------")
print("")
print("Finish")

----------------------------------------------------------------------
----------------------------------------------------------------------
Opening file:  PlateSolved_Calib_ngc-869_LIGHT_2021-02-02T21-07-35Z_R_120s_-4.9C_E_0007.fits
Starting aperture photometry...
----------------------------------------------------------------------
----------------------------------------------------------------------
Opening file:  PlateSolved_Calib_ngc-869_LIGHT_2021-02-02T20-43-07Z_G_120s_-4.9C_E_0018.fits
Starting aperture photometry...
----------------------------------------------------------------------
----------------------------------------------------------------------
Opening file:  PlateSolved_Calib_ngc-869_LIGHT_2021-02-02T20-33-55Z_G_120s_-4.9C_E_0014.fits
Starting aperture photometry...
----------------------------------------------------------------------
----------------------------------------------------------------------
Opening file:  PlateSolved_Calib_ngc-869_LIGHT_2021-03-12

In [9]:
G_data = np.column_stack((G_ID, G_RA, G_DEC, G_Flux))
I_data = np.column_stack((I_ID, I_RA, I_DEC, I_Flux))     
R_data = np.column_stack((R_ID, R_RA, R_DEC, R_Flux)) 

#Converting filter-specific arrays to dataframes, sorting them, removing dupliacte source enrties and saving as a .csv file.
Gdf = pd.DataFrame(G_data, columns = ['ID', 'Right-Ascension' , 'Declination', 'Flux']) 
Gdf = Gdf.sort_values(by = ['ID'], ascending = True )
Gdf = Gdf.groupby(['ID']).mean(['Flux'])
Gdf = Gdf.round({'Right-Ascension': 3, 'Declination': 3})
Gdf = Gdf.sort_values(by = (['Flux']), ascending = True )
Gdf.to_csv("Test.csv")

Idf = pd.DataFrame(I_data, columns = ['ID', 'Right-Ascension' , 'Declination', 'Flux']) 
Idf = Idf.sort_values(by = ['ID'], ascending = True )
Idf = Idf.groupby(['ID']).mean(['Flux'])
Idf = Idf.round({'Right-Ascension': 3, 'Declination': 3})

Rdf = pd.DataFrame(R_data, columns = ['ID', 'Right-Ascension' , 'Declination', 'Flux']) 
Rdf = Rdf.sort_values(by = ['ID'], ascending = True )
Rdf = Rdf.groupby(['ID']).mean(['Flux'])
Rdf = Rdf.round({'Right-Ascension': 3, 'Declination': 3})

def Flux_to_mag(Flux):
    return -2.5*np.log10(Flux)

for n in range(len(Gdf['Flux'])):
    if Gdf['Flux'][n+1] < 6 or Idf['Flux'][n+1] < 6 or Rdf['Flux'][n+1] < 6:
        Gdf = Gdf.drop([n+1])
        Idf = Idf.drop([n+1])
        Rdf = Rdf.drop([n+1])

G_mag = Flux_to_mag(Gdf['Flux'])
Gdf['G_mag'] = G_mag

I_mag = Flux_to_mag(Idf['Flux'])
Idf['I_mag'] = I_mag

R_mag = Flux_to_mag(Rdf['Flux'])
Rdf['R_mag'] = R_mag

Gdf.to_csv(os.path.join(local_dir, "2. " + cluster_dir + "_G_data.csv"))
Idf.to_csv(os.path.join(local_dir, "2. " + cluster_dir + "_I_data.csv"))
Rdf.to_csv(os.path.join(local_dir, "2. " + cluster_dir + "_R_data.csv"))